# Linear Regression in Gluon
### Generating Data Sets

In [1]:
!pip install mxnet

Traceback (most recent call last):
  File "/home/iserina/.local/bin/pip", line 7, in <module>
    from pip._internal import main
ModuleNotFoundError: No module named 'pip._internal'


In [2]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = nd.array([2, -3.4])
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = nd.dot(features, true_w) + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

### Reading Data

In [3]:
from mxnet.gluon import data as gdata

batch_size = 10
# Combine the features and labels of the training data
dataset = gdata.ArrayDataset(features, labels)
# Randomly reading mini-batches
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

# Read a batch to see how it works
for X, y in data_iter:
    print(X, y)
    break


[[-0.48212412  0.13752407]
 [ 0.52979773 -2.436552  ]
 [-1.7312217   0.24892013]
 [-0.00972697  0.28969124]
 [ 0.15112837 -1.3817692 ]
 [ 2.6056926   0.58565795]
 [-0.9827446   0.75879306]
 [-0.152714   -0.6237566 ]
 [ 0.2963081   0.8711869 ]
 [-0.26519665  1.5181851 ]]
<NDArray 10x2 @cpu(0)> 
[ 2.7621648  13.54595    -0.09809604  3.1917932   9.198681    7.4225383
 -0.36555594  6.0042424   1.8301574  -1.4937832 ]
<NDArray 10 @cpu(0)>


### Define the Model

In [4]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

### Initialize Model Parameters

- Initialize weight parameter by a normal distribution with a mean of 0 and standard deviation of 0.01.
- The bias parameter is initialized to zero by default.

In [5]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

### Define the Loss Function

In [6]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()  # The squared loss is also known as the L2 norm loss

### Define the Optimization Algorithm

In [7]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

### Training

In [8]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.040421
epoch 2, loss: 0.000151
epoch 3, loss: 0.000051


### Evaluate 

In [9]:
w = net[0].weight.data()
print('Error in estimating w', true_w.reshape(w.shape) - w)
b = net[0].bias.data()
print('Error in estimating b', true_b - b)
print(w)
print(b)

Error in estimating w 
[[ 0.00010836 -0.00010824]]
<NDArray 1x2 @cpu(0)>
Error in estimating b 
[0.00076962]
<NDArray 1 @cpu(0)>

[[ 1.9998916 -3.3998919]]
<NDArray 1x2 @cpu(0)>

[4.19923]
<NDArray 1 @cpu(0)>


#Linear Regression with Keras

In [10]:
import pandas as pd

import numpy as np
import keras

from matplotlib import pyplot as plt

from keras.layers import Input, Dense
from keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

%matplotlib inline

Using TensorFlow backend.
/home/iserina/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iserina/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iserina/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iserina/.local/lib/python3.6/site-packages/tensorflow/py

In [11]:
inputs = Input(shape=(2,))
preds = Dense(1,activation='linear')(inputs)
X_=features.asnumpy()
y_=labels.asnumpy()
model = Model(inputs=inputs,outputs=preds)
sgd=keras.optimizers.SGD()
model.compile(optimizer=sgd ,loss='mse',metrics=['mse'])
model.fit(X_,y_, batch_size=batch_size, epochs=num_epochs, shuffle=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
1000/1000 [==============================] - 0s 245us/step - loss: 9.0932 - mean_squared_error: 9.0932
Epoch 2/3
1000/1000 [==============================] - 0s 92us/step - loss: 0.1676 - mean_squared_error: 0.1676
Epoch 3/3
1000/1000 [==============================] - 0s 92us/step - loss: 0.0033 - mean_squared_error: 0.0033


In [12]:
weights, biases = model.layers[1].get_weights()
print('Error in estimating w', true_w.asnumpy() - weights.reshape(true_w.shape))
print('Error in estimating b', true_b - biases)
print("weights=", weights)
print("biases=",biases)


Error in estimating w [ 0.00148404 -0.00882387]
Error in estimating b [0.01266575]
weights= [[ 1.998516 ]
 [-3.3911762]]
biases= [4.187334]


In [13]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [14]:
ynew = model.predict(X_)
train_l = squared_loss(ynew, y_)
train_l.mean()

0.00016998946